Import useful libs here




In [ ]:
!pip install musicgeneration==0.10.3
import musicgeneration.songs as mgen

%tensorflow_version 2.x
import tensorflow as tf 

import numpy as np

download and manipulate dataset (music in abc notation) using custom builded lib

In [ ]:
songs = mgen.load_training_data()

Found 817 songs in text


In [ ]:
songs_joined = "\n\n".join(songs)
unique_symbols = sorted(set(songs_joined))


create numerical represintation of characters in the song

In [ ]:
symbol2id = {symbol:id for id,symbol in enumerate(unique_symbols)}

id2symbol = np.array(unique_symbols)


create array of numbers for each song in the dataset

In [ ]:
def vectorize_song(initial_songs):
  return np.array([symbol2id[character] for character in initial_songs])

In [ ]:
vectorized_song = vectorize_song(songs_joined)

create training songs batches for further training ml model

In [ ]:
def get_batch(vectorized_songs, seq_length, batch_size):
  """
  param vectorized_songs : numerical represintation of songs in the dataset
  seq_len : count of characters in the song
  batch_size : count of songs in the training example
  """
  n = vectorized_songs.shape[0] - 1
  # randomly choose the starting indices for the examples in the training batch
  idx = np.random.choice(n-seq_length, batch_size)

  '''ist of input sequences for the training batch'''
  input_batch = [vectorized_songs[i : i+seq_length] for i in idx]
  '''list of output sequences for the training batch'''
  output_batch = [vectorized_songs[i+1 : i+seq_length+1] for i in idx]

  # x_batch, y_batch provide the true inputs and targets for network training
  x_batch = np.reshape(input_batch, [batch_size, seq_length])
  y_batch = np.reshape(output_batch, [batch_size, seq_length])
  return x_batch, y_batch


define LSTM RNN network

In [ ]:
def LSTM(rnn_units): 
  return tf.keras.layers.LSTM(
    rnn_units, 
    return_sequences=True, 
    recurrent_initializer='glorot_uniform',
    recurrent_activation='sigmoid',
    stateful=True,
  )

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    # Layer 1: Embedding layer to transform indices into dense vectors 
    #   of a fixed embedding size
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),

    # Layer 2: LSTM with `rnn_units` number of units. 
    LSTM(rnn_units), 

    # Layer 3: Dense (fully-connected) layer that transforms the LSTM output
    #   into the vocabulary size. 
    tf.keras.layers.Dense(vocab_size)
  ])

  return model

# Build a simple model with default hyperparameters. You will get the 
#   chance to change these later.
model = build_model(len(unique_symbols), embedding_dim=256, rnn_units=1024, batch_size=32)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (32, None, 256)           21248     
                                                                 
 lstm (LSTM)                 (32, None, 1024)          5246976   
                                                                 
 dense (Dense)               (32, None, 83)            85075     
                                                                 
Total params: 5,353,299
Trainable params: 5,353,299
Non-trainable params: 0
_________________________________________________________________


In [ ]:
x, y = get_batch(vectorized_song, seq_length=100, batch_size=32)
pred = model(x)
print("Input shape:      ", x.shape, " # (batch_size, sequence_length)")
print("Prediction shape: ", pred.shape, "# (batch_size, sequence_length, vocab_size)")

Input shape:       (32, 100)  # (batch_size, sequence_length)
Prediction shape:  (32, 100, 83) # (batch_size, sequence_length, vocab_size)


In [ ]:
sampled_indices = tf.random.categorical(pred[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([45,  8, 53, 34, 29, 78, 46, 26, 22, 10, 41, 76, 73, 13, 27, 77, 50,
        4, 30, 12, 42, 67, 78, 26, 47, 60,  0,  0, 50,  0, 34, 55, 23, 82,
       19, 69, 40, 62,  0, 30, 10, 46, 34, 80, 55, 11, 45, 82,  8, 36, 10,
       47, 35, 60, 44, 70, 75, 80, 54,  3, 58, 50, 32, 34, 44, 44, 52, 41,
       80,  6, 30, 35, 34, 29, 57, 51,  1, 82, 10, 78, 60, 56, 20, 48, 61,
       14, 57, 66, 10, 77, 42, 39, 49, 19, 74, 59, 11, 45, 73,  2])

In [ ]:
print("Input: \n", repr("".join(id2symbol[x[0]])))
print()
print("Next Char Predictions: \n", repr("".join(id2symbol[sampled_indices])))

Input: 
 '2 G2:|!\n(3Bcd|g2dB GBdg|e2cA FGAg|f2ed ^cdeg|(3fgf (3efe dcBA|!\nG2DG BGBd|c2AB cdef|gdBG FAdc|B2G2 G'

Next Char Predictions: 
 'T,]IDwUA:.Pur1BvY#E0QlwAVe\n\nY\nI_<|7nOg\nE.UIy_/T|,K.VJeSoty^"cYGISS[Py(EJIDbZ |.wea8Wf2bk.vQNX7sd/Tr!'


In [ ]:

'''TODO: define the loss function to compute and return the loss between
    the true labels and predictions (logits). Set the argument from_logits=True.'''
def compute_loss(labels, logits):
  loss = tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
  return loss

'''TODO: compute the loss using the true next characters from the example batch 
    and the predictions from the untrained model several cells above'''
example_batch_loss = compute_loss(y, pred)
# example_batch_loss = compute_loss('''TODO''', '''TODO''') # TODO

print("Prediction shape: ", pred.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (32, 100, 83)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.4192057


In [ ]:
### Hyperparameter setting and optimization ###

# Optimization parameters:
num_training_iterations = 2000  # Increase this to train longer
batch_size = 4  # Experiment between 1 and 64
seq_length = 100  # Experiment between 50 and 500
learning_rate = 5e-3  # Experiment between 1e-5 and 1e-1

# Model parameters: 
vocab_size = len(vocab)
embedding_dim = 256 
rnn_units = 1024  # Experiment between 1 and 2048

# Checkpoint location: 
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "my_ckpt")

In [ ]:
### Define optimizer and training operation ###
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)

optimizer = tf.keras.optimizers.Adam(learning_rate)

@tf.function
def train_step(x, y): 
  # Use tf.GradientTape()
  with tf.GradientTape() as tape:
  
    y_hat = model(x)
  
    loss = compute_loss(y, y_hat)
 
    grads = tape.gradient(loss, model.trainable_variables) # TODO
    
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

##################
# Begin training!#
##################

history = []
plotter = mdl.util.PeriodicPlotter(sec=2, xlabel='Iterations', ylabel='Loss')
if hasattr(tqdm, '_instances'): tqdm._instances.clear() # clear if it exists

for iter in tqdm(range(num_training_iterations)):

  # Grab a batch and propagate it through the network
  x_batch, y_batch = get_batch(vectorized_songs, seq_length, batch_size)
  loss = train_step(x_batch, y_batch)

  # Update the progress bar
  history.append(loss.numpy().mean())
  plotter.plot(history)

  # Update the model with the changed weights!
  if iter % 100 == 0:     
    model.save_weights(checkpoint_prefix)
    
# Save the trained model and the weights
model.save_weights(checkpoint_prefix)
